In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
train_dir = '../Alzheimer_s Dataset/Alzheimer_s Dataset/train'
test_dir = '../Alzheimer_s Dataset/Alzheimer_s Dataset/test'
image_size = (128, 128)
batch_size = 32

In [3]:
data_generator = ImageDataGenerator(rescale = 1./255)

In [4]:
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 5121 images belonging to 4 classes.


In [5]:
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 1279 images belonging to 4 classes.


In [6]:
model = Sequential([
    Conv2D(32,(3,3),input_shape=(image_size[0],image_size[1],3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64,(3,3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

c:\Users\dodym\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0],-1)
train_labels = train_generator.classes

c:\Users\dodym\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 97ms/step


In [8]:
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0],-1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 94ms/step


In [9]:
cb_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, verbose=0)
cb_classifier.fit(train_features, train_labels)

In [10]:
test_preds = cb_classifier.predict(test_features)

In [11]:
test_accuracy = accuracy_score(test_labels, test_preds)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 53.79%
